**Based on Lanaro: Python High Performance**

# Futures

Futures are a more convenient pattern that can be used to keep track of the results of
asynchronous calls. In the preceding code, we saw that rather than returning values, we
accept callbacks and pass the results when they are ready. It is interesting to note that, so
far, there is no easy way to track the status of the resource

A future is an abstraction that helps us keep track of the requested resources and that we
are waiting to become available. In Python, you can find a future implementation in the
**concurrent.futures.Future** class. A Future instance can be created by calling its
constructor with no arguments:

In [1]:
from concurrent.futures import Future
import threading

In [2]:
fut = Future()

In [3]:
print(fut)

<Future at 0x7f5a1c537ac8 state=pending>


In [4]:
fut.set_result("Hello")

In [5]:
print(fut)

<Future at 0x7f5a1c537ac8 state=finished returned str>


In [6]:
fut.result()

'Hello'

You can see that once we set the result, the Future will report that the task is finished and
can be accessed using the Future.result method. It is also possible to subscribe a callback
to a future so that, as soon as the result is available, the callback is executed. To attach a
callback, it is sufficient to pass a function to the Future.add_done_callback method

In [7]:
fut = Future()
fut.add_done_callback(lambda future: print("The result is: " + future.result(),flush=True))

In [8]:
fut.set_result("Hello")

The result is: Hello


To get a grasp on how futures can be used in practice, we will adapt the network_request_async function to use futures. 

The idea is that, this time, instead of returning nothing, we return a Future that will keep track of the result for us.


In [9]:
def network_request_async(number):
        future = Future()
        result = {"success": True, "result": number ** 2}
        timer = threading.Timer(5.0, lambda: future.set_result(result))
        timer.start()
        return future

In [10]:
fut1 = network_request_async(2)
fut2 = network_request_async(3)
fut3 = network_request_async(4)

In [11]:
print(fut1.result())
print(fut2.result())
print(fut3.result())

{'success': True, 'result': 4}
{'success': True, 'result': 9}
{'success': True, 'result': 16}


Example: 
From a function calling an asyncronuous function that gives back a future object as a result.
Then calling a callback function (on_done_future) when the future object gets the results    

In [12]:
def fetch_square(number):
        fut = network_request_async(number)

        def on_done_future(future):
            response = future.result()
            if response["success"]:
                print("Result is: {}".format(response["result"]))
    
        fut.add_done_callback(on_done_future)

In [13]:
fetch_square(2)
fetch_square(3)
fetch_square(4)

Result is: 4
Result is: 9
Result is: 16
